# A Menu Converter Project

## Introduction

In this script, we automate the process of converting PDF menu to images, and finally into a structured Excel spreadsheet using the OpenAI GPT model.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
directory = '/content/drive/MyDrive/RAG Course/5_PDF_To_Image_To_Excel_Using_OpenAI'

## PDF To Image Converter

### Install and Import Required Library

In [3]:
!pip install PyMuPDF Pillow

* **PyMuPDF**: A Python binding for MuPDF, which allows for PDF and image file processing.
* **Pillow**: A Python Imaging Library that adds image processing capabilities to your Python interpreter.

In [4]:
import fitz # PyMuPDF
from PIL import Image # Pillow
import os

* `fitz` provides functions to read and manipulate PDF files.
* `Image` from `PIL` allows us to create and modify images.
* `os` helps in interacting with the operating system, such as reading files and directories.

### Define The PDF To JPG Conversion Function

We define a function pdf_to_jpg that converts all PDF files in the specified directory to JPG images.

In [5]:
def pdf_to_jpg(directory):
  # Iterate over all files in the specified directory
  for filename in os.listdir(directory):
    # Check if the file is a PDF
    if filename.endswith('.pdf'):
      # Construct the full file path
      pdf_path = os.path.join(directory, filename)
      # Open the PDF file
      pdf_document = fitz.open(pdf_path)
      # Iterate over each page in the PDF
      for page_number in range(len(pdf_document)):
        # Get the page by its index
        page = pdf_document.load_page(page_number)
        # Render the page as apixmap (an in-memory image)
        pixmap = page.get_pixmap()

        # Construct the output image file path
        image_path = os.path.join(
            directory,
            f"images/{os.path.splitext(filename)[0]}_page{page_number+1}.jpg"
        )
        # Create the pixmap as a JPG image
        img = Image.frombytes("RGB", [pixmap.width, pixmap.height], pixmap.samples)
        # Save the image to the specified path
        img.save(image_path, "JPEG")

  # Print a message when all conversions are done
  print("All PDF files have been converted")

In this function:

* We loop through all files in the directory and select those that end with `.pdf`.
* Each PDF is opened using `fitz.open()`.
* We iterate through each page of the PDF.
* Each page is rendered to a pixmap using `page.get_pixmap()`.
* The pixmap is converted to an image using `Image.frombytes()`.
* The image is saved as a JPG file in the same directory.

In [6]:
# Apply the function
pdf_to_jpg(directory)

All PDF files have been converted


## Image To Excel Converter

### Install and Import Required Library

In [7]:
from openai import OpenAI
import os
import base64
from IPython.display import Image, display, Markdown
import pandas as pd

In [8]:
# Retrieve the OpenAI API key from Collab's user data
from google.colab import userdata
open_ai_key = userdata.get('openai_api')

In [9]:
# Setup OpenAI client and specify the model to use
MODEL = "gpt-4o"
client = OpenAI(api_key=open_ai_key)

### Deine The System Prompt

We define a detailed system prompt that instructs the GPT model on how to convert the menu images into a structured Excel format.


In [10]:
# Define the system prompt with detailed instructions
system_prompt = """
Convert the menu image to a structured excel sheet format following the provided template and instructions.
This assistant converts restaurant or cafe menu data into a structured Excel sheet that adheres to a specific template.
The template includes categories, subcategories, item names, prices, descriptions, and more, ensuring data consistency.
This assistant helps users fill out each row correctly, following the detailed instructions provided.

Overview:
- Each row in the Excel spreadsheet represents a unique item, categorized under a category or subcategory.
- Category and subcategory names are repeated for items within the same subcategory.
- Certain columns are left blank when not applicable, such as subcategory details for items directly under a category.
- Item details, including names, prices, and descriptions, must be unique for each entry.
- Uploaded menu content will be appended to the existing menu without deleting any current entries.

Columns Guide:

Column Name                    | Description                               | Accepted Values           | Example
-------------------------------|-------------------------------------------|---------------------------|-----------------------
CategoryTitlePt (Column A)      | Category names in Portuguese              | Text, 256 characters max  | Bebidas
CategoryTitleEn (Column B) (Optional) | English translations of category titles | Text, 256 characters max  | Beverages
SubcategoryTitlePt (Column C) (Optional) | Subcategory titles in Portuguese | Text, 256 characters max or blank | Sucos
SubcategoryTitleEn (Column D) (Optional) | English translations of subcategory titles | Text, 256 characters max or blank | Juices
ItemNamePt (Column E)           | Item names in Portuguese                  | Text, 256 characters max  | Água Mineral
ItemNameEn (Column F) (Optional) | English translations of item names | Text, 256 characters max or blank | Mineral Water
ItemPrice (Column G)          | Price of each item without currency symbol  | Text                      | 2.50 or 2,50
Calories (Column H) (Optional) | Caloric content of each item              | Numeric                   | 150
PortionSize (Column I)        | Portion size for each item in units        | Text                      | 500ml, 1, 2-3
Availability (Column J) (Optional) | Current availability of the item     | Numeric: 1 for Yes, 0 for No | 1
ItemDescriptionPt (Column K) (Optional) | Detailed description in Portuguese | Text, 500 characters max  | Contains essential minerals
ItemDescriptionEn (Column L) (Optional) | Detailed description in English | Text, 500 characters max  | Contains essential minerals

Notes:
- Ensure all data entered follows the specified formats to maintain database integrity.
- Review the data for accuracy and consistency before submitting the Excel sheet.
"""


In [11]:
# Change the current working directory to the image directory
os.chdir(directory)
IMAGE_DIR = f"{directory}/images"

def encode_image(image_path):
  # Open the image file in binary mode and encode it in Base64
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')

# Process images in the directory
image_files = sorted([f for f in os.listdir(IMAGE_DIR) if f.lower().endswith(('.jpg', '.png', '.jpeg'))])
image_files


['menu_page1.jpg',
 'menu_page2.jpg',
 'menu_page3.jpg',
 'menu_page4.jpg',
 'menu_page5.jpg',
 'menu_page6.jpg',
 'menu_page7.jpg']

Encoding images in Base64 allows us to include image data directly in our API requests without relying on external URLs.

This code scans the directory for files ending with `.png`, `.jpg`, or `.jpeg`, ensuring we only process image files relevant to our task.

### Define The Image To Excel Conversion Function

We prompt the user to input a name for the new Excel file where the extracted data will be saved.

We loop through each image file, encode it, send it to the OpenAI API for processing, and parse the response to populate our DataFrame.

In [12]:
# Prompt the user for the excel file name
new_excel_file_name = input("Enter the name for the new Excel file to be created (without extension): ")
EXCEL_PATH = os.path.join(directory, f"{new_excel_file_name}.xlsx")

# Create the Pandas DataFrame
df = pd.DataFrame(columns=['CategoryTitlePt', 'CategoryTitleEn', 'SubcategoryTitlePt', 'SubcategoryTitleEn',
                           'ItemNamePt', 'ItemNameEn', 'ItemPrice', 'Calories', 'PortionSize', 'Availability',
                           'ItemDescriptionPt', 'ItemDescriptionEn'])

for image in image_files:
  # Retrieve and encode the image
  image_path = os.path.join(IMAGE_DIR, image)
  image_data = encode_image(image_path)

  # Adding a flag for the headers
  headers_added = False

  # Use GPT-4o to analyze and convert the image
  response = client.chat.completions.create(
      model=MODEL,
      messages=[
          {"role": "system", "content": system_prompt},
          {"role": "user", "content": [
              {"type": "text", "text": "Convert this menu image to a structured Excel Sheet Format."},
              {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_data}"}}
          ]}
      ],
      temperature=0
  )

  for row in response.choices[0].message.content.split('\n'):
    # Ensure the data is a row and not a header
    if row.startswith('|') and not row.startswith('|-'):
      columns = [col.strip() for col in row.split('|')[1:-1]]
      if len(columns) == len(df.columns):
        if 'CategoryTitlePt' in columns:
            headers_added = True
            continue
        # Skip the row
        if headers_added and 'CategoryTitlePt' in columns:
          continue
        new_row = pd.Series(columns, index=df.columns)
        df = pd.concat([df, pd.DataFrame([new_row])], ignore_index=True)
      else:
        print(f"Skipping row {row}")

df.to_excel(EXCEL_PATH, index=False)
print(f"Excel file saved at: {EXCEL_PATH}")

Enter the name for the new Excel file to be created (without extension): menu_as_excel
Excel file saved at: /content/drive/MyDrive/RAG Course/5_PDF_To_Image_To_Excel_Using_OpenAI/menu_as_excel.xlsx


In this loop:

* **Encoding the Image**: Each image is encoded in Base64 format using the `encode_image` function.
* **API Request**: We send the encoded image along with the prompt to the OpenAI API using `client.chat.completions.create`.
* **Temperature Parameter**: We set `temperature=0` to make the output deterministic, ensuring consistent formatting.
* **Response Parsing**: The API response is expected to be in a Markdown table format. We parse each line, checking if it's a data row.
* **Data Extraction**: We extract the columns, check if they match the expected number of DataFrame columns, and append them to the DataFrame.
* **Error Handling**: If a row doesn't match the expected format, we print a message and skip it.


In [13]:
df

,CategoryTitlePt,CategoryTitleEn,SubcategoryTitlePt,SubcategoryTitleEn,ItemNamePt,ItemNameEn,ItemPrice,Calories,PortionSize,Availability,ItemDescriptionPt,ItemDescriptionEn
0,ENTRADAS,STARTERS,,,Azeitonas,Olives,1.60,,,1,,
1,ENTRADAS,STARTERS,,,Cesto de Pão,Bread Basket,1.80,,,1,,
2,ENTRADAS,STARTERS,,,Creme de Legumes,Cream of Vegetables,3.00,,,1,,
3,PARA PARTILHAR,TO SHARE,,,Nachos & Guacamole,Nachos & Guacamole,7.50,,,1,,
4,PARA PARTILHAR,TO SHARE,,,REGATTA Tuna,Marinated Tuna,12.90,,,1,Atum Fresco Marinado,Marinated Tuna
...,...,...,...,...,...,...,...,...,...,...,...,...
74,Digestivos & Licores,Digestives & Liqueurs,,,Moscatel Favaios,Moscatel Favaios,2.00,,,1,,
75,Digestivos & Licores,Digestives & Liqueurs,,,Licor Beirão,Beirão Liqueur,3.00,,,1,,
76,Digestivos & Licores,Digestives & Liqueurs,,,Vinho do Porto,Port Wine,3.00,,,1,,
77,Digestivos & Licores,Digestives & Liqueurs,,,Whisky,Whisky,5.00,,,1,,


After processing all images, we save the populated DataFrame to an Excel file.

This script demonstrates how to automate the extraction of structured data from menu images using the OpenAI GPT model.

By converting menu images into a standardized Excel format, we facilitate easier data management and analysis for restaurant or cafe menus.

The use of the OpenAI API for image-to-text conversion streamlines the data entry process, reducing manual effort and potential errors.